In [1]:
%pylab inline
import numpy as np
import matplotlib.pyplot as plt
import time

Populating the interactive namespace from numpy and matplotlib


# Analisis evolutivo de insectos transgenicos de la especie <i>Rhodnius prolixus</i> en el tratamiento de la enfermedad de Chagas

## Introduccion

Realizado en conjunto con <strong>Nancy Estefania Ruiz Uribe</strong>. Notebook para describir un modelo evolutivo para los vectores con las diferentes bacterias modificadas en el marco del proyecto de <i>Biologia Sintetica</i> para el semestre 2015-1 dictado por <i>Juan Manuel Pedraza</i>. 

Nos interesa observar la evolucion en el tiempo de las poblaciones de vectores.

Nos basamos en el Algoritmo de Gillepsie desarrollado para la tarea 3.

Tenemos poblaciones de:
<ul>
<li>Vectores <strong>sin bacterias</strong>  modificadas</li>
<li>Vectores con <strong>E.Coli</strong> modificada</li>
<li>Vectores con <strong>R.Rhodni</strong> modificada</li>
<li>Vectores con <strong>Salmonella Enterica</strong> modificada</li>
</ul>

##Constantes

Encontrar constantes (valores numericos) es todo un reto. En efecto para encontrar estas constantes se debe realizar un importante trabajo experimental y, muchas veces, este no ha sido realizado aun.

Nuestras constante seran entonces, en un primero momento, proporcionales a las constantes de las bacterias (fitness).

In [ ]:
#Constantes

In [ ]:
def evento_gillepsie(num_rna,num_prot,t,k_r,k_p,gamma_p,gamma_r):
    e1=np.random.random() #se crea un numero aleatorio entre 0 y 1 mediante una distribucion uniforme 
    #Normalicemos las constantes
    k_total=k_r+k_p*num_rna+gamma_p*num_prot+gamma_r*num_rna
    paso_t=np.random.exponential()/(k_total)
    t_nuevo=t+paso_t
    #Ahora mimremos que tipo de evento sucedio
    if(e1<(k_r/k_total)):#Habemus rna
        num_rna_nuevo=num_rna+1.0
        num_prot_nuevo=num_prot
    elif((e1<((k_r+gamma_r*num_rna)/k_total)) and (e1>=(k_r/k_total))):#Destruimos Rna
        num_rna_nuevo=num_rna-1.0
        num_prot_nuevo=num_prot
    elif((e1<((k_r+k_p*num_rna+gamma_r*num_rna)/k_total)) and (e1>=((k_r+gamma_r*num_rna)/k_total))):#HAbemus proteina
        num_rna_nuevo=num_rna
        num_prot_nuevo=num_prot+1.0
    else:#Destruimos proteina
        num_rna_nuevo=num_rna
        num_prot_nuevo=num_prot-1.0
    return num_rna_nuevo, num_prot_nuevo,t_nuevo

def simulacion_gillepsie(num_cells,t_limite,delta_t,k_r,k_p,gamma_r,gamma_p):
    contador_cells=0.0
    num_puntos=int(t_limite/delta_t)
    matriz_rna=np.zeros((num_cells,num_puntos))
    matriz_prot=np.zeros((num_cells,num_puntos))
    
    while (contador_cells<num_cells):
        num_rna=0.0
        num_prot=0.0
        t_total=0.0
        t_walk=[]
        r_walk=[]
        prot_walk=[]
        while (t_total<t_limite):
            num_rna, num_prot,t_total=evento_gillepsie(num_rna,num_prot,t_total,k_r,k_p,gamma_p,gamma_r)
            t_walk.append(t_total)
            r_walk.append(num_rna)
            prot_walk.append(num_prot)
        #Ahora se deben reformar los arreglos por los tiempos
        t_final = np.linspace(0, t_limite-delta_t, t_limite/delta_t)
        r_final = np.zeros(len(t_final))
        prot_final = np.zeros(len(t_final))
        pos=0
        for i in range (len(t_final)):
            while((t_walk[pos] < t_final[i])&((pos+1)<len(prot_walk))):
                pos=pos+1
            r_final[i] = r_walk[pos]
            prot_final[i] = prot_walk[pos]
        matriz_rna[contador_cells]=r_final
        matriz_prot[contador_cells]=prot_final
        contador_cells=contador_cells+1.0
    
    #Ahora calculamos la media y la desviacion
    r_mean=np.zeros(num_puntos)
    r_std=np.zeros(num_puntos)
    prot_mean=np.zeros(num_puntos)
    prot_std=np.zeros(num_puntos)
    for i in range(num_puntos):
        r_mean[i]=np.mean(matriz_rna[:,i])
        r_std[i]=np.std(matriz_rna[:,i])
        prot_mean[i]=np.mean(matriz_prot[:,i])
        prot_std[i]=np.std(matriz_prot[:,i])
        
        
    return t_final, matriz_rna, matriz_prot, r_mean, r_std,prot_mean,prot_std